
## Restricted Boltzman Machines


In [8]:

import numpy as np


In [9]:

class RBM:
    
    def __init__(self, n_visible, n_hidden, learning_rate=0.1):
        self.num_visible   = n_visible
        self.num_hidden    = n_hidden
        self.learning_rate = learning_rate

        self.weights      = np.random.randn(num_hidden, num_visible) * 0.1  
        self.hidden_bias  = np.zeros(num_hidden)  
        self.visible_bias = np.zeros(num_visible)  

    def sigmoid(self, x):
        return 1.0 / (1.0 + np.exp(-x))

    def sample_probabilities(self, probabilities):
        
        return np.random.binomial( 1, probabilities )

    def forward_pass(self, visible):
        hidden_activations   = np.dot(visible, self.weights.T) + self.hidden_bias
        hidden_probabilities = self.sigmoid(hidden_activations)
        return hidden_probabilities

    def backward_pass(self, hidden):
        visible_activations   = np.dot(hidden, self.weights) + self.visible_bias
        visible_probabilities = self.sigmoid( visible_activations ) 
        return visible_probabilities


    def train(self, data, epochs=10):
        
        for epoch in range(epochs):
            epoch_error = 0
            
            for sample in data:
                # Pos phase
                positive_hidden_probs  = self.forward_pass(sample)
                positive_hidden_states = self.sample_probabilities(positive_hidden_probs)
                positive_associations  = np.outer(positive_hidden_states, sample)

                # Neg phase
                negative_visible_probs = self.backward_pass(positive_hidden_states)
                negative_hidden_probs  = self.forward_pass(negative_visible_probs)
                negative_associations  = np.outer(negative_hidden_probs, negative_visible_probs)

                # Update weights and biases
                self.weights = self.weights + self.learning_rate * (positive_associations - negative_associations)
                self.visible_bias = self.visible_bias + self.learning_rate * (sample - negative_visible_probs)
                self.hidden_bias  = self.hidden_bias  + self.learning_rate * (positive_hidden_probs - negative_hidden_probs)

                epoch_error = epoch_error + np.sum(  (sample - negative_visible_probs) ** 2  )

            print(f"Epoch is {epoch + 1}, Reconstruction Error: {epoch_error:.4f}")

    def reconstruct(self, input_data):
        hidden                = self.forward_pass( input_data )
        reconstructed         = self.backward_pass( hidden    )
        return reconstructed





In [10]:

data = np.array([
    [1,1, 1, 0, 1, 0],
    [0, 1, 0, 1, 0, 1],
    [1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 0],
    [0, 1, 0, 1, 0, 1]
])


In [11]:

num_visible = data.shape[1]
num_hidden  = 4
rbm = RBM(n_visible=num_visible, n_hidden=num_hidden, learning_rate=0.1)


rbm.train(data, epochs=10)



Epoch is 1, Reconstruction Error: 7.7999
Epoch is 2, Reconstruction Error: 7.4151
Epoch is 3, Reconstruction Error: 7.8316
Epoch is 4, Reconstruction Error: 7.3105
Epoch is 5, Reconstruction Error: 7.4262
Epoch is 6, Reconstruction Error: 7.3444
Epoch is 7, Reconstruction Error: 7.0804
Epoch is 8, Reconstruction Error: 7.1209
Epoch is 9, Reconstruction Error: 7.5254
Epoch is 10, Reconstruction Error: 7.4025


In [12]:


test_sample   = np.array([1, 0, 1, 0, 0, 0])  # New input
reconstructed = rbm.reconstruct(test_sample)

print("\nOriginal Input:    ", test_sample)
print("Reconstructed Output:", np.round(reconstructed))



Original Input:     [1 0 1 0 0 0]
Reconstructed Output: [0. 1. 1. 1. 0. 0.]


In [13]:

def hamming_distance(a, b):
    """
    Calculate the Hamming distance between two strings.
    """

    ## arr1 = np.array(list(str1), dtype=int)
    
    # XOR to find differing bits, then count the number of 1s
    return np.count_nonzero(a != b)


In [14]:

distance = hamming_distance(test_sample, reconstructed)
distance


6